# Speech Emotion recognition

The database is from berlin emotinal speech data http://emodb.bilderbar.info/docu/

From the document attached, the last second character represents the emotion in German

classify data by the name

In [18]:
from shutil import copyfile
import glob
import sys

emotion_class_map = {'W' : 0, 'L' : 1, 'E' : 2, 'A' : 3, 'F' : 4, 'T' : 5, 'N' : 6}
emotions = ["anger", "boredom", "disgust", "fear", "happy", "sadness", "neutral"]
participants = glob.glob("./wav/*")
for source in participants:
    cur_emo_ind = emotion_class_map[source[-6]]
    dest = "./data/%s/%s" %(emotions[cur_emo_ind], source[-11:]) 
    copyfile(source, dest) #Copy file

In [44]:
from python_speech_features.base import mfcc, fbank, logfbank, ssc
import scipy.io.wavfile as wav
import numpy as np

cepCount = 13 #no of MFCC coefficients
nfeatures = 7 #features per coefficient

emotion_class_map = {'W' : 0, 'L' : 1, 'E' : 2, 'A' : 3, 'F' : 4, 'T' : 5, 'N' : 6}
emotions = ["anger", "boredom", "disgust", "fear", "happy", "sadness", "neutral"]


def audio_read(datafs):
    (data, fs) = wav.read(datafs)
    ceps = mfcc(fs, numcep = cepCount)
    feat2 = ssc(fs, samplerate = 16000, winlen = 0.025, winstep=0.01, 
                nfilt = 26, nfft = 512, lowfreq = 0, highfreq = None, preemph=0.97)
    ls = []
    for i in range(ceps.shape[1]):
        temp = ceps[:,i]
        dtemp = np.gradient(temp)
        lfeatures  = [np.mean(temp), np.var(temp), np.amax(temp), np.amin(temp), 
        np.var(dtemp), np.mean(temp[0:temp.shape[0]/2]), np.mean(temp[temp.shape[0]/2:temp.shape[0]])]
        temp2 = np.array(lfeatures)
        ls.append(temp2)
    
    ls2 = []
    for i in range(feat2.shape[1]):
        temp = feat2[:,i]
        dtemp = np.gradient(temp)
        lfeatures = [np.mean(temp), np.var(temp), np.amax(temp), np.amin(temp), 
        np.var(dtemp), np.mean(temp[0:temp.shape[0]/2]), np.mean(temp[temp.shape[0]/2:temp.shape[0]])]
        temp2 = np.array(lfeatures)
        ls2.append(temp2)
    
    source = np.array(ls).flatten()
    source = np.append(source, np.array(ls2).flatten())
    return source

def data_load():
    features = []
    labels = []
    count = 0
    participants = glob.glob("./wav/*")
    for source in participants:
        cur_emo_ind = emotion_class_map[source[-6]]
        features.append(audio_read(source))
        labels.append(cur_emo_ind)
        count += 1
    return features, labels

print "Start loading..."
X, y = data_load()
print "Start dumping..."
X = np.array(X)
y = np.array(y)
X.dump("X.dat")
y.dump("y.dat")
print "Done."

Start loading...
Start dumping...
Done.


Load pre processed data, X and y.

In [66]:
X = np.load("X.dat")
y = np.load("y.dat")

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler(copy=True, with_mean=True, with_std=True)
X2 = scaler.fit_transform(X)

Making train and test set and used different classifer to test the performance

In [77]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X2, y, test_size = 0.2)

from sklearn.svm import SVC
clf = SVC(C=5, kernel='rbf', degree=3, decision_function_shape='ovr', random_state=None)

# clf = SVC(verbose=True, cache_size = 700)

clf.fit(X_train, y_train)

print "Training Accuracy %.2f\n" % (clf.score(X_train, y_train))
print "Testing Accuracy %.2f\n" % (clf.score(X_test, y_test))

Training Accuracy 1.00

Testing Accuracy 0.92



In [68]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X2, y, test_size = 0.2)

from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, y_train)


print "Training Accuracy %.2f\n" % (clf.score(X_train, y_train))
print "Testing Accuracy %.2f\n" % (clf.score(X_test, y_test))

Training Accuracy 1.00

Testing Accuracy 0.79



In [71]:
from sklearn.externals import joblib
joblib.dump(clf, 'audio_SVC_model.pkl')

['audio_SVC_model.pkl']